In [ ]:
# Load packages
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
import sportsdataverse as sdv
import pyodbc


Load data from sportsdataverse

In [7]:
# Load data
df_schedule = sdv.nfl.load_nfl_schedule(seasons=range(2015,2022))

100%|██████████| 7/7 [00:01<00:00,  3.63it/s]


In [8]:
# Filter for Regular Season
df_schedule = df_schedule[df_schedule['game_type'] == 'REG']
df_schedule.head()

Data wrangling

In [10]:
# Calculate winning team
df_schedule['win_team'] = np.where(df_schedule['home_result'] >= 0, df_schedule['home_team'], df_schedule['away_team'])
df_schedule['win_score'] = np.where(df_schedule['home_result'] >= 0, df_schedule['home_score'], df_schedule['away_score'])

df_schedule['loss_team'] = np.where(df_schedule['home_result'] < 0, df_schedule['home_team'], df_schedule['away_team'])
df_schedule['loss_score'] = np.where(df_schedule['home_result'] < 0, df_schedule['home_score'], df_schedule['away_score'])

In [11]:
# Select required columns
df_schedule[['game_id', 'season', 'away_team', 'home_team', 'home_result', 'win_team', 'win_score', 'loss_team', 'loss_score']]

,game_id,season,away_team,home_team,home_result,win_team,win_score,loss_team,loss_score
0,2015_01_PIT_NE,2015,PIT,NE,7,NE,28,PIT,21
1,2015_01_IND_BUF,2015,IND,BUF,13,BUF,27,IND,14
2,2015_01_GB_CHI,2015,GB,CHI,-8,GB,31,CHI,23
3,2015_01_KC_HOU,2015,KC,HOU,-7,KC,27,HOU,20
4,2015_01_CAR_JAX,2015,CAR,JAX,-11,CAR,20,JAX,9
...,...,...,...,...,...,...,...,...,...
1871,2021_18_NYJ_BUF,2021,NYJ,BUF,17,BUF,27,NYJ,10
1872,2021_18_SF_LA,2021,SF,LA,-3,SF,27,LA,24
1873,2021_18_NE_MIA,2021,NE,MIA,9,MIA,33,NE,24
1874,2021_18_CAR_TB,2021,CAR,TB,24,TB,41,CAR,17


In [12]:
# Rename columns
df_wins = df_schedule[['game_id', 'season', 'win_team', 'win_score']].rename(columns = {'win_team' : 'team', 'win_score' : 'points'})
df_wins['result'] = 1
df_wins.head()

,game_id,season,team,points,result
0,2015_01_PIT_NE,2015,NE,28,1
1,2015_01_IND_BUF,2015,BUF,27,1
2,2015_01_GB_CHI,2015,GB,31,1
3,2015_01_KC_HOU,2015,KC,27,1
4,2015_01_CAR_JAX,2015,CAR,20,1
...,...,...,...,...,...
1871,2021_18_NYJ_BUF,2021,BUF,27,1
1872,2021_18_SF_LA,2021,SF,27,1
1873,2021_18_NE_MIA,2021,MIA,33,1
1874,2021_18_CAR_TB,2021,TB,41,1


In [13]:
# Rename columns for losses
df_losses = df_schedule[['game_id', 'season', 'loss_team', 'loss_score']].rename(columns = {'loss_team' : 'team', 'loss_score' : 'points'})
df_losses['result'] = 0
df_losses

,game_id,season,team,points,result
0,2015_01_PIT_NE,2015,PIT,21,0
1,2015_01_IND_BUF,2015,IND,14,0
2,2015_01_GB_CHI,2015,CHI,23,0
3,2015_01_KC_HOU,2015,HOU,20,0
4,2015_01_CAR_JAX,2015,JAX,9,0
...,...,...,...,...,...
1871,2021_18_NYJ_BUF,2021,NYJ,10,0
1872,2021_18_SF_LA,2021,LA,24,0
1873,2021_18_NE_MIA,2021,NE,24,0
1874,2021_18_CAR_TB,2021,CAR,17,0


In [14]:
df_results  = pd.concat([df_wins, df_losses])
df_results

,game_id,season,team,points,result
0,2015_01_PIT_NE,2015,NE,28,1
1,2015_01_IND_BUF,2015,BUF,27,1
2,2015_01_GB_CHI,2015,GB,31,1
3,2015_01_KC_HOU,2015,KC,27,1
4,2015_01_CAR_JAX,2015,CAR,20,1
...,...,...,...,...,...
1871,2021_18_NYJ_BUF,2021,NYJ,10,0
1872,2021_18_SF_LA,2021,LA,24,0
1873,2021_18_NE_MIA,2021,NE,24,0
1874,2021_18_CAR_TB,2021,CAR,17,0


In [15]:
df_results_agg = df_results.groupby(['season', 'team']).agg({'points' : 'sum', 'result' : 'sum'}).rename(columns = {'result' : 'wins'}).reset_index()
df_results_agg

,season,team,points,wins
0,2015,ARI,489,13
1,2015,ATL,339,8
2,2015,BAL,328,5
3,2015,BUF,379,8
4,2015,CAR,500,15
...,...,...,...,...
219,2021,SEA,395,7
220,2021,SF,427,10
221,2021,TB,511,13
222,2021,TEN,419,12


In [16]:
df_results_agg['last_season'] = df_results_agg['season']-1
df_results_merged = df_results_agg.merge(df_results_agg, left_on = ['last_season','team'], right_on = ['season', 'team'], suffixes = ('', '_lastseason'))
df_results_merged = df_results_merged[['season', 'team', 'points', 'wins', 'points_lastseason', 'wins_lastseason']]
df_results_merged

,season,team,points,wins,points_lastseason,wins_lastseason
0,2016,ARI,418,8,489,13
1,2016,ATL,540,11,339,8
2,2016,BAL,343,8,328,5
3,2016,BUF,399,7,379,8
4,2016,CAR,369,6,500,15
...,...,...,...,...,...,...
184,2021,SEA,395,7,459,12
185,2021,SF,427,10,376,6
186,2021,TB,511,13,492,11
187,2021,TEN,419,12,491,11


In [17]:
df_train = df_results_merged[df_results_merged['season'] < 2021].reset_index(drop=True)
df_test = df_results_merged[df_results_merged['season'] == 2021].reset_index(drop=True)

In [18]:
train_x = df_train[['points_lastseason', 'wins_lastseason']]
train_y = df_train[['wins']]
lr_fit = LinearRegression().fit(train_x, train_y)

In [19]:
lr_fit.coef_

array([[0.01176864, 0.17216294]])

In [20]:
test_x = df_test[['points_lastseason', 'wins_lastseason']]

In [21]:
test_predictions = lr_fit.predict(test_x)
df_test['wins_prediction'] = test_predictions
df_test.head()

,season,team,points,wins,points_lastseason,wins_lastseason,wins_prediction
0,2021,ARI,449,11,410,8,8.559330
1,2021,ATL,313,7,396,4,7.705918
2,2021,BAL,387,8,468,11,9.758400
3,2021,BUF,483,11,501,13,10.491091
4,2021,CAR,304,5,350,5,7.336723


In [22]:
df_test['error'] = df_test['wins_prediction'] - df_test['wins']

errors_array = df_test['error'].to_numpy()
squared_errors = np.square(errors_array)
mean_squared_error = squared_errors.mean()
root_mean_squared_error = np.sqrt(mean_squared_error)
root_mean_squared_error

2.2554371124390684

In [23]:
df_new = df_test[['season', 'team', 'points', 'wins']].copy()
df_new['season'] = df_new['season']+1
df_new = df_new.rename(columns = {'points' : 'points_lastseason', 'wins' : 'wins_lastseason'})
df_new['wins'] = np.nan
df_new.head()

,season,team,points_lastseason,wins_lastseason,wins
0,2022,ARI,449,11,NaN
1,2022,ATL,313,7,NaN
2,2022,BAL,387,8,NaN
3,2022,BUF,483,11,NaN
4,2022,CAR,304,5,NaN


In [24]:
df_new_preds = df_new.copy()
df_new_preds = df_new_preds.drop('wins', axis = 1)
new_x = df_new_preds[['points_lastseason', 'wins_lastseason']]
new_predictions = lr_fit.predict(new_x)
df_new_preds['predicted_wins'] = new_predictions
df_new_preds.head()

,season,team,points_lastseason,wins_lastseason,predicted_wins
0,2022,ARI,449,11,9.534796
1,2022,ATL,313,7,7.245610
2,2022,BAL,387,8,8.288652
3,2022,BUF,483,11,9.934929
4,2022,CAR,304,5,6.795366


In [25]:
# Create connection to AdventureWorksDW
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=localhost\SQLEXPRESS;'
                      'Database=AdventureWorksDW;'
                      'Trusted_Connection=yes;')

# Create python cursor
cursor = conn.cursor()

In [26]:
#cursor.execute("DROP TABLE nfl_wins_pred")

In [27]:
# Create results table
cursor.execute(
    
""" 
CREATE TABLE nfl_wins_pred(
    season				INTEGER NOT NULL
    ,team				VARCHAR(3) NOT NULL
    ,points_lastseason  INTEGER NOT NULL
    ,wins_lastseason    INTEGER NOT NULL
    ,predicted_wins     NUMERIC(6,2)  NOT NULL
    )    
"""
)

In [28]:
# Insert Dataframe into SQL Server:
for i in range(len(df_new_preds)):
    
    cursor.execute("INSERT INTO nfl_wins_pred (season, team, points_lastseason, wins_lastseason, predicted_wins) values(?,?,?,?,?)"
    , int(df_new_preds.iloc[i][0])
    , str(df_new_preds.iloc[i][1])
    , int(df_new_preds.iloc[i][2])
    , int(df_new_preds.iloc[i][3])
    , float(df_new_preds.iloc[i][4])
    )
conn.commit()

In [29]:
# Close database connection
cursor.close()
print('Insert Complete')

Insert Complete
